In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
# funcion de selección de ROI del objeto a seguir.
def selectROI(event, x, y, flags, param):
    global frame, roiPts, inputMode
    # si estamos en modo de selección de ROI, si clickeo el mouse y aun no tenemos 2 puntos definidos,
    # entonces actualizamos la lista de puntos de posiciones x,y donde se clikeo el mouse y dibujamos un circulo.
    if inputMode and event == cv2.EVENT_LBUTTONDOWN and len(roiPts) < 2:
        roiPts.append((x,y))
        cv2.circle(frame, (x,y), 4, (0,0,255), 2)
        cv2.imshow("frame", frame)

In [3]:
frame = None
roiPts = []
inputMode = False

In [4]:
global frame, roiPts, inputMode
cap = cv2.VideoCapture(0)
cv2.namedWindow("frame")
cv2.setMouseCallback("frame", selectROI)
track_window = None
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 10)
while True:
    # obtenemos lectura de la camara web
    ret, frame = cap.read()

    # chequeamos que haya lectura
    if ret==True:
        # si ya tenemos un ROI seleccionado:
        if track_window is not None:
            # pasamos a HSV y hacemos CamShift
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
            
            ret, track_window = cv2.CamShift(dst, track_window, term_crit)
            pts = np.int0(cv2.boxPoints(ret))
            cv2.polylines(frame, [pts], True, (0, 255, 0), 2)

        cv2.imshow("frame", frame)
        key = cv2.waitKey(1) & 0xFF
        # Si apreto g y no tengo 2 roipts, genero los roipts.
        if key == ord("g") and len(roiPts) < 2:
            inputMode = True
            frame2 = frame.copy()
            while len(roiPts) < 2:
                cv2.imshow("frame", frame)
                cv2.waitKey(0)

            roiPts = np.array(roiPts)
            s = roiPts.sum(axis = 1)
            a = roiPts[np.argmin(s)]
            b = roiPts[np.argmax(s)]
            roi = frame2[a[1]:b[1], a[0]:b[0]]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

            # histograma HSV y definimos ROI
            hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            # mascara para el histograma vista en clase
            mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            #histograma sobre esa mascara
            roi_hist = cv2.calcHist([roi], [0], mask, [180], [0, 180])
            roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
            track_window = (a[0], a[1], b[0], b[1])
    # si apretamos Escape terminamos
        elif key==27:
            break
            
#reseteo Roipts para volver a empezar y libero la cámara.
roiPts=[]
cap.release()
cv2.destroyAllWindows()